# Agenda
1. Grouping and pivot tables
2. More with multi-indexes (e.g., stack and unstack)
3. Joining, merging, and concatenating
4. Working with text



In [3]:
import pandas as pd

filename = 'taxi.csv'

df = pd.read_csv(filename,
                usecols=['VendorID', 'passenger_count', 'trip_distance',
                         'total_amount', 'payment_type'])

In [4]:
df.head()

,VendorID,passenger_count,trip_distance,payment_type,total_amount
0,2,1,1.63,2,17.80
1,2,1,0.46,1,8.30
2,2,1,0.87,1,11.00
3,2,1,2.13,1,17.16
4,1,1,1.40,2,10.30


In [5]:
# I want to know how much people paid, on average (mean) for their taxi rides

df['total_amount'].mean()

np.float64(17.552472247224728)

In [7]:
# I want to know how much people paid, on average, for their taxi rides where there were 0 passengers

(
    df.loc[
        df['passenger_count'] == 0,
        'total_amount'
       ]
    .mean()
)

np.float64(25.57)

In [8]:
# I want to know how much people paid, on average, for their taxi rides where there were 1 passenger

(
    df.loc[
        df['passenger_count'] == 1,
        'total_amount'
       ]
    .mean()
)

np.float64(17.368569446371584)

In [9]:
# I want to know how much people paid, on average, for their taxi rides where there were 2 passengers

(
    df.loc[
        df['passenger_count'] == 2,
        'total_amount'
       ]
    .mean()
)

np.float64(18.406306169078444)

# DRY -- don't repeat yourself!

If you're running the same query for each distinct value in a particular column, there is a better way to do this -- to do grouping, which we run via the `groupby` method.

The idea is:
- Choose a categorical column, i.e., one with a limited number of distinct values
- We choose a numeric column, i.e., one on which we'll want to perform the calculation
- We choose an aggregation method, i.e., one which takes many values and returns a single value

The syntax for `groupby` is:

    df.groupby(CATEGORICAL)[NUMERIC].AGGFUNC()

The result will be a series. The index for this series will be the distinct values of `CATEGORICAL`, sorted in ascending order. The values will be the result of invoking `AGGFUNC` on each subset of `NUMERIC`.

In [10]:
df.groupby('passenger_count')['total_amount'].mean()

passenger_count
0    25.570000
1    17.368569
2    18.406306
3    17.994704
4    18.881648
5    17.211269
6    17.401355
Name: total_amount, dtype: float64

Any time that you ask, "What was the value of X for each value of Y," you're asking a `groupby` question:

- Sales per region
- Sales per product
- Salary per age
- Expenses per household

# What aggregation methods are there?

- `min`
- `max`
- `mean`
- `std`
- `median`
- `quantile`
- `sum`
- `count` -- how many non-`NaN` values are there?
- `idxmin`
- `idxmax`
- `value_counts`

In [11]:
df.groupby('passenger_count')['total_amount'].idxmin()

passenger_count
0    5097
1    5719
2    9052
3     603
4    1014
5    5087
6    7509
Name: total_amount, dtype: int64

In [12]:
df.groupby('passenger_count')['total_amount'].value_counts()

passenger_count  total_amount
0                14.75             1
                 36.39             1
1                7.30            210
                 7.80            186
                 6.80            179
                                ... 
6                63.41             1
                 63.55             1
                 70.01             1
                 72.92             1
                 83.12             1
Name: count, Length: 1749, dtype: int64

# Exercise: Taxi grouping

1. We're going to run a bunch of queries using `groupby` on the NYC taxi data from January 2020. (This is in the larger zipfile that I asked you to download. The filename is `nyc_taxi_2020-01.csv`.)
2. What was the mean `total_amount` for each value of `passenger_count`?
3. What was the max `total_amount` for each value of `passenger_count`?
4. Create a new column, `tip_percentage`, which is the result of taking the `tip_amount` and finding its percentage of `fare_amount`. Get the mean `tip_percentage` per `passenger_count`.
5. Compare the mean and median `total_amount` for each value of `payment_type`.

In [15]:
filename = '/Users/reuven/Courses/Current/Data/nyc_taxi_2020-01.csv'

df = pd.read_csv(filename)

/var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_24291/111017737.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [16]:
!ls -lh $filename

-rw-r--r-- 1 reuven staff 567M Jun  4  2021 /Users/reuven/Courses/Current/Data/nyc_taxi_2020-01.csv


In [17]:
df = pd.read_csv(filename, low_memory=False)

In [18]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [19]:
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [20]:
# What was the mean total_amount for each value of passenger_count?

df.groupby('passenger_count')['total_amount'].mean()


passenger_count
0.0    18.059724
1.0    18.343110
2.0    19.050504
3.0    18.736862
4.0    19.128092
5.0    18.234443
6.0    18.367962
7.0    71.143103
8.0    58.197059
9.0    81.244211
Name: total_amount, dtype: float64

In [21]:
df['passenger_count'].value_counts()

passenger_count
1.0    4547226
2.0     946423
3.0     250234
5.0     225693
6.0     132154
4.0     123470
0.0     114302
7.0         29
9.0         19
8.0         17
Name: count, dtype: int64

In [22]:
# What was the max total_amount for each value of passenger_count?

df.groupby('passenger_count')['total_amount'].max()


passenger_count
0.0     435.42
1.0    4268.30
2.0     617.30
3.0     499.56
4.0     730.30
5.0     384.66
6.0     352.30
7.0     101.30
8.0     121.31
9.0     140.06
Name: total_amount, dtype: float64

In [26]:
df.groupby('passenger_count')['total_amount'].min()


passenger_count
0.0    -128.30
1.0   -1242.30
2.0    -177.80
3.0    -169.80
4.0    -730.30
5.0    -130.80
6.0     -65.30
7.0       8.30
8.0       8.80
9.0      11.76
Name: total_amount, dtype: float64

In [25]:
df.loc[df['total_amount'] == 4268.30].iloc[0]

VendorID                                 2.0
tpep_pickup_datetime     2020-01-21 15:38:33
tpep_dropoff_datetime    2020-01-27 13:43:40
passenger_count                          1.0
trip_distance                           1.57
RatecodeID                               1.0
store_and_fwd_flag                         N
PULocationID                             186
DOLocationID                             152
payment_type                             2.0
fare_amount                           4265.0
extra                                    0.0
mta_tax                                  0.5
tip_amount                               0.0
tolls_amount                             0.0
improvement_surcharge                    0.3
total_amount                          4268.3
congestion_surcharge                     2.5
Name: 4049543, dtype: object

In [30]:
# Create a new column, tip_percentage, which is the result of taking the tip_amount and finding its percentage of fare_amount. 
# Get the mean tip_percentage per passenger_count.

df['tip_percentage'] = df['tip_amount'] / df['fare_amount']

df.groupby('passenger_count')['tip_percentage'].mean()

passenger_count
0.0         inf
1.0         inf
2.0         inf
3.0    0.187235
4.0         inf
5.0    0.200383
6.0         inf
7.0    0.524173
8.0    0.138806
9.0    0.131651
Name: tip_percentage, dtype: float64

In [33]:
df.loc[df['total_amount'] == 0].iloc[0]

VendorID                                 1.0
tpep_pickup_datetime     2020-01-01 00:28:00
tpep_dropoff_datetime    2020-01-01 00:28:35
passenger_count                          1.0
trip_distance                            0.0
RatecodeID                               1.0
store_and_fwd_flag                         N
PULocationID                             166
DOLocationID                             166
payment_type                             3.0
fare_amount                              0.0
extra                                    0.0
mta_tax                                  0.0
tip_amount                               0.0
tolls_amount                             0.0
improvement_surcharge                    0.0
total_amount                             0.0
congestion_surcharge                     0.0
tip_percentage                           NaN
Name: 2318, dtype: object

In [39]:
df.dropna(subset=['tip_amount', 'fare_amount', 'tip_percentage']).groupby('passenger_count')['tip_percentage'].mean()

passenger_count
0.0         inf
1.0         inf
2.0         inf
3.0    0.187235
4.0         inf
5.0    0.200383
6.0         inf
7.0    0.524173
8.0    0.138806
9.0    0.131651
Name: tip_percentage, dtype: float64

In [41]:
df.loc[df['fare_amount'] != 0].groupby('passenger_count')['tip_percentage'].mean()

passenger_count
0.0    0.193764
1.0    0.209198
2.0    0.193316
3.0    0.187235
4.0    0.175032
5.0    0.200383
6.0    0.197588
7.0    0.524173
8.0    0.138806
9.0    0.131651
Name: tip_percentage, dtype: float64

In [42]:
# method chaining

(
    
    df
    .loc[df['fare_amount'] != 0]     # only keep rows where fare amount isn't 0
    .groupby('passenger_count')['tip_percentage'].mean()
)

passenger_count
0.0    0.193764
1.0    0.209198
2.0    0.193316
3.0    0.187235
4.0    0.175032
5.0    0.200383
6.0    0.197588
7.0    0.524173
8.0    0.138806
9.0    0.131651
Name: tip_percentage, dtype: float64

In [43]:
# Compare the mean and median total_amount for each value of payment_type.

df.groupby('payment_type')['total_amount'].mean()

payment_type
1.0    19.602178
2.0    15.516222
3.0     9.933257
4.0     0.890626
5.0     0.000000
Name: total_amount, dtype: float64

In [44]:
df.groupby('payment_type')['total_amount'].median()

payment_type
1.0    14.8
2.0    11.8
3.0     9.3
4.0     0.3
5.0     0.0
Name: total_amount, dtype: float64

In [46]:
df.groupby('payment_type')['total_amount'].agg(['mean', 'median'])

,mean,median
payment_type,,
1.0,19.602178,14.8
2.0,15.516222,11.8
3.0,9.933257,9.3
4.0,0.890626,0.3
5.0,0.000000,0.0


In [47]:
# we see here how we can run groupby on a categorical column
# what if I want to groupby on *two* categoricals?
# typically, it'll be hierarchical
# - country + region
# - department + product
# - year + month

# let's get the mean amount paid 
# for each passenger_count + payment_type combination

df.groupby(['passenger_count', 'payment_type'])['total_amount'].mean()

passenger_count  payment_type
0.0              1.0             19.169661
                 2.0             15.080724
                 3.0             14.753550
                 4.0             15.009711
1.0              1.0             19.479882
                 2.0             15.272954
                 3.0              9.479660
                 4.0              0.501119
                 5.0              0.000000
2.0              1.0             20.196789
                 2.0             16.277862
                 3.0             12.061866
                 4.0              0.310638
3.0              1.0             19.839760
                 2.0             16.203684
                 3.0             11.231576
                 4.0              0.022131
4.0              1.0             20.463335
                 2.0             16.687550
                 3.0             11.233845
                 4.0              1.112188
5.0              1.0             19.285617
                 2.0    

In [49]:
# after we perform the groupby, we can use xs to retrieve only those results
# where payment_type == 1

df.groupby(['passenger_count', 'payment_type'])['total_amount'].mean().xs(1, level='payment_type')

passenger_count
0.0    19.169661
1.0    19.479882
2.0    20.196789
3.0    19.839760
4.0    20.463335
5.0    19.285617
6.0    19.406970
7.0    77.342174
8.0    53.255000
9.0    82.345556
Name: total_amount, dtype: float64

In [ ]:
# method chaining here:

(
    df
    .groupby(['passenger_count', 'payment_type'])['total_amount'].mean()
    .xs(1, level='payment_type')
)

In [53]:
df.set_index(['passenger_count', 'payment_type'])

VendorID tpep_pickup_datetime  \
passenger_count payment_type                                  
1.0             1.0                1.0  2020-01-01 00:28:15   
                1.0                1.0  2020-01-01 00:35:39   
                1.0                1.0  2020-01-01 00:47:41   
                1.0                1.0  2020-01-01 00:55:23   
                2.0                2.0  2020-01-01 00:01:58   
...                                ...                  ...   
NaN             NaN                NaN  2020-01-31 22:51:00   
                NaN                NaN  2020-01-31 22:10:00   
                NaN                NaN  2020-01-31 22:50:07   
                NaN                NaN  2020-01-31 22:25:53   
                NaN                NaN  2020-01-31 22:44:00   

                             tpep_dropoff_datetime  trip_distance  RatecodeID  \
passenger_count payment_type                                                    
1.0             1.0            2020-01-01 00:33:03           1.20         1.0   
                1.0            2020-01-01 00:43:04           1.20         1.0   
                1.0            2020-01-01 00:53:52           0.60         1.0   
                1.0            2020-01-01 01:00:14           0.80         1.0   
                2.0            2020-01-01 00:04:16           0.00         1.0   
...                                            ...            ...         ...   
NaN             NaN            2020-01-31 23:22:00           3.24         NaN   
                NaN            2020-01-31 23:26:00          22.13         NaN   
                NaN            2020-01-31 23:17:57          10.51         NaN   
                NaN            2020-01-31 22:48:32           5.49         NaN   
                NaN            2020-01-31 23:06:00          11.60         NaN   

                             store_and_fwd_flag  PULocationID  DOLocationID  \
passenger_count payment_type                                                  
1.0             1.0                           N           238           239   
                1.0                           N           239           238   
                1.0                           N           238           238   
                1.0                           N           238           151   
                2.0                           N           193           193   
...                                         ...           ...           ...   
NaN             NaN                         NaN           237           234   
                NaN                         NaN           259            45   
                NaN                         NaN           137           169   
                NaN                         NaN            50            42   
                NaN                         NaN           179           205   

                              fare_amount  extra  mta_tax  tip_amount  \
passenger_count payment_type                                            
1.0             1.0                  6.00   3.00      0.5        1.47   
                1.0                  7.00   3.00      0.5        1.50   
                1.0                  6.00   3.00      0.5        1.00   
                1.0                  5.50   0.50      0.5        1.36   
                2.0                  3.50   0.50      0.5        0.00   
...                                   ...    ...      ...         ...   
NaN             NaN                 17.59   2.75      0.5        0.00   
                NaN                 46.67   2.75      0.5        0.00   
                NaN                 48.85   2.75      0.0        0.00   
                NaN                 27.17   2.75      0.0        0.00   
                NaN                 54.56   2.75      0.5        0.00   

                              tolls_amount  improvement_surcharge  \
passenger_count payment_type                                        
1.0             1.0                   0.

In [54]:
# what about calculating on multiple columns?
# if we want, we can pass a list of numeric columns on which to calculate

df.groupby('passenger_count')[['total_amount', 'trip_distance']].mean()

,total_amount,trip_distance
passenger_count,,
0.0,18.059724,2.689548
1.0,18.343110,2.811050
2.0,19.050504,3.001117
3.0,18.736862,2.930363
4.0,19.128092,2.980372
5.0,18.234443,2.850356
6.0,18.367962,2.906041
7.0,71.143103,3.589655
8.0,58.197059,2.960000


In [55]:
# if we pass a single numeric column, then we get a series

df.groupby('passenger_count')['total_amount'].mean()

passenger_count
0.0    18.059724
1.0    18.343110
2.0    19.050504
3.0    18.736862
4.0    19.128092
5.0    18.234443
6.0    18.367962
7.0    71.143103
8.0    58.197059
9.0    81.244211
Name: total_amount, dtype: float64

In [56]:
# if we pass a single numeric column inside of a one-element list, then we get a data frame

df.groupby('passenger_count')[['total_amount']].mean()

,total_amount
passenger_count,
0.0,18.059724
1.0,18.343110
2.0,19.050504
3.0,18.736862
4.0,19.128092
5.0,18.234443
6.0,18.367962
7.0,71.143103
8.0,58.197059


# Summarize so far

We can run `.groupby` on:
- a categorical column
- a numerical column
- with an aggregation method

*BUT* we can actually pass:
- a list of categorical columns
- a list of numeric columns
- more than one aggregation method

Doing of these gives us a data frame, rather than a series as a result.



# what if I were to:

- groupby both `passenger_count` and `payment_type`
- calculate on `trip_distance` and `total_amount`

In [57]:
df.groupby(['passenger_count', 'payment_type'])[['trip_distance', 'total_amount']].mean()

trip_distance  total_amount
passenger_count payment_type                             
0.0             1.0                2.725365     19.169661
                2.0                2.593740     15.080724
                3.0                2.509309     14.753550
                4.0                2.789210     15.009711
1.0             1.0                2.851244     19.479882
                2.0                2.701837     15.272954
                3.0                2.368599      9.479660
                4.0                2.474345      0.501119
                5.0                0.000000      0.000000
2.0             1.0                3.024937     20.196789
                2.0                2.930926     16.277862
                3.0                3.281866     12.061866
                4.0                3.105367      0.310638
3.0             1.0                2.931454     19.839760
                2.0                2.923253     16.203684
                3.0                3.219514     11.231576
                4.0                2.938678      0.022131
4.0             1.0                2.977180     20.463335
                2.0                2.980704     16.687550
                3.0                3.171519     11.233845
                4.0                3.296442      1.112188
5.0             1.0                2.889323     19.285617
                2.0                2.756536     15.385476
                3.0                0.737833     -8.708917
                4.0                0.350608     -8.882376
6.0             1.0                2.937371     19.406970
                2.0                2.831080     15.556227
                3.0                1.071713     -3.133370
                4.0                0.593472     -2.484444
7.0             1.0                3.811304     77.342174
                2.0                2.740000     47.380000
8.0             1.0                2.187857     53.255000
                2.0                6.563333     81.260000
9.0             1.0                3.060000     82.345556
                2.0                7.900000     61.420000

In [58]:
# can we run more than one aggregation method?

df.groupby(['passenger_count', 'payment_type'])[['trip_distance', 'total_amount']].agg(['median', 'mean'])

trip_distance           total_amount           
                                    median      mean       median       mean
passenger_count payment_type                                                
0.0             1.0                  1.500  2.725365       14.750  19.169661
                2.0                  1.400  2.593740       11.800  15.080724
                3.0                  1.100  2.509309       10.300  14.753550
                4.0                  1.000  2.789210        9.300  15.009711
1.0             1.0                  1.600  2.851244       14.800  19.479882
                2.0                  1.480  2.701837       11.800  15.272954
                3.0                  0.900  2.368599        8.800   9.479660
                4.0                  1.000  2.474345       -2.705   0.501119
                5.0                  0.000  0.000000        0.000   0.000000
2.0             1.0                  1.680  3.024937       15.300  20.196789
                2.0                  1.540  2.930926       12.300  16.277862
                3.0                  1.200  3.281866       10.800  12.061866
                4.0                  1.200  3.105367       -4.800   0.310638
3.0             1.0                  1.640  2.931454       14.800  19.839760
                2.0                  1.570  2.923253       12.300  16.203684
                3.0                  1.200  3.219514       10.800  11.231576
                4.0                  1.200  2.938678       -4.800   0.022131
4.0             1.0                  1.680  2.977180       15.340  20.463335
                2.0                  1.600  2.980704       12.800  16.687550
                3.0                  1.200  3.171519       11.300  11.233845
                4.0                  1.400  3.296442        6.550   1.112188
5.0             1.0                  1.670  2.889323       14.800  19.285617
                2.0                  1.520  2.756536       11.800  15.385476
                3.0                  0.420  0.737833       -7.300  -8.708917
                4.0                  0.220  0.350608       -6.800  -8.882376
6.0             1.0                  1.660  2.937371       14.800  19.406970
                2.0                  1.530  2.831080       12.300  15.556227
                3.0                  0.500  1.071713       -7.300  -3.133370
                4.0                  0.135  0.593472       -6.300  -2.484444
7.0             1.0                  0.020  3.811304       80.800  77.342174
                2.0                  1.840  2.740000       53.980  47.380000
8.0             1.0                  0.000  2.187857       42.360  53.255000
                2.0                  1.210  6.563333       83.300  81.260000
9.0             1.0                  0.000  3.060000       95.800  82.345556
                2.0                  7.900  7.900000       61.420  61.420000

# Exercise: Olympic data 

1. Create a data frame with the file `olympic_athlete_events.csv`.
2. What was the mean height per team in years 1960 and onward?
3. What were the mean height and weight per team in basketball and speed skating?
4. What were the mean and median age per country, in years 1980 and onward?

In [59]:
# this file contains information about every Olympic athlete and event until 2020

filename = '/Users/reuven/Courses/Current/Data/olympic_athlete_events.csv'

!head $filename

"ID","Name","Sex","Age","Height","Weight","Team","NOC","Games","Year","Season","City","Sport","Event","Medal"
"1","A Dijiang","M",24,180,80,"China","CHN","1992 Summer",1992,"Summer","Barcelona","Basketball","Basketball Men's Basketball",NA
"2","A Lamusi","M",23,170,60,"China","CHN","2012 Summer",2012,"Summer","London","Judo","Judo Men's Extra-Lightweight",NA
"3","Gunnar Nielsen Aaby","M",24,NA,NA,"Denmark","DEN","1920 Summer",1920,"Summer","Antwerpen","Football","Football Men's Football",NA
"4","Edgar Lindenau Aabye","M",34,NA,NA,"Denmark/Sweden","DEN","1900 Summer",1900,"Summer","Paris","Tug-Of-War","Tug-Of-War Men's Tug-Of-War","Gold"
"5","Christine Jacoba Aaftink","F",21,185,82,"Netherlands","NED","1988 Winter",1988,"Winter","Calgary","Speed Skating","Speed Skating Women's 500 metres",NA
"5","Christine Jacoba Aaftink","F",21,185,82,"Netherlands","NED","1988 Winter",1988,"Winter","Calgary","Speed Skating","Speed Skating Women's 1,000 metres",NA
"5","Christine Jacoba Aaftink","F",25,1

In [60]:
df = pd.read_csv(filename)
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [68]:
# What was the mean height per team in years 1960 and onward?

(
    df
    .loc[df['Year'] >= 1960]    # keep only 1960+ years
    .groupby('Team')['Height'].mean()
)

Team
Puerto Rico-1            196.000000
Nadine                   190.000000
Ireland-1                189.666667
Serbia-2                 189.000000
Puerto Rico-2            188.000000
Salamander               187.666667
Serbia and Montenegro    187.511401
India-1                  187.500000
Bingo                    187.000000
Ireland-2                187.000000
Name: Height, dtype: float64

In [71]:
# What were the mean height and weight per team in basketball and speed skating?


(
    df
    .loc[(df['Sport'] == 'Basketball') | (df['Sport'] == 'Speed Skating')]
    .groupby('Team')[['Height', 'Weight']].mean()
)

,Height,Weight
Team,,
Angola,191.788732,90.225352
Argentina,198.272727,101.033333
Australia,188.704762,85.373377
Austria,173.774510,70.901961
Belarus,178.086957,69.673913
...,...,...
United States,181.281134,76.589617
Uruguay,187.577778,86.372093
Venezuela,196.428571,101.294118


In [72]:
# we can use the "isin" method

(
    df
    .loc[df['Sport'].isin(['Basketball', 'Speed Skating'])]
    .groupby('Team')[['Height', 'Weight']].mean()
)

,Height,Weight
Team,,
Angola,191.788732,90.225352
Argentina,198.272727,101.033333
Australia,188.704762,85.373377
Austria,173.774510,70.901961
Belarus,178.086957,69.673913
...,...,...
United States,181.281134,76.589617
Uruguay,187.577778,86.372093
Venezuela,196.428571,101.294118


In [73]:
# What were the mean and median age per country, in years 1980 and onward?

(
    df
    .loc[df['Year'] >= 1980]
    .groupby('Team')['Age'].agg(['mean', 'median'])
)

,mean,median
Team,,
Afghanistan,23.000000,23.0
Albania,25.230769,23.0
Algeria,24.346743,24.0
American Samoa,27.216216,26.0
Andorra,23.283871,22.0
...,...,...
Yugoslavia,23.535286,23.0
Yugoslavia-1,25.250000,24.5
Yugoslavia-2,24.250000,25.0


In [74]:
df = pd.read_csv('/Users/reuven/Courses/Current/Data/nyc_taxi_2020-01.csv', low_memory=False)

# I want to know:

- for each payment type
- for each number of passengers
- what was the mean total_amount?

One reasonable way to depict this would be in a table (or a data frame):

- The rows (index) would be different payment types
- The columns would be different numbers of passengers
- The values would be taken from `total_amount`
- We would run `mean` on the combination at the intersection

This, in the Pandas world, is known as a "pivot table"!

To create a pivot table, we need to specify:

- What categorical column will we use for the `index` (rows)?
- What categorical column will we use for the `columns`?
- What numeric column will we use for the values?
- What aggregate method will we invoke?

In [75]:
# this pivot_table method is the best way to create a pivot table
# there is also a "pivot" method, but it only works if there's one value for each row-column combination
# (it cannot handle aggregation methods)

df.pivot_table(index='payment_type',
               columns='passenger_count',
               values='total_amount',
               aggfunc='mean')

passenger_count,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
payment_type,,,,,,,,,,
1.0,19.169661,19.479882,20.196789,19.839760,20.463335,19.285617,19.406970,77.342174,53.255,82.345556
2.0,15.080724,15.272954,16.277862,16.203684,16.687550,15.385476,15.556227,47.380000,81.260,61.420000
3.0,14.753550,9.479660,12.061866,11.231576,11.233845,-8.708917,-3.133370,NaN,NaN,NaN
4.0,15.009711,0.501119,0.310638,0.022131,1.112188,-8.882376,-2.484444,NaN,NaN,NaN
5.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
df.groupby(['passenger_count', 'payment_type'])['total_amount'].mean()

passenger_count  payment_type
0.0              1.0             19.169661
                 2.0             15.080724
                 3.0             14.753550
                 4.0             15.009711
1.0              1.0             19.479882
                 2.0             15.272954
                 3.0              9.479660
                 4.0              0.501119
                 5.0              0.000000
2.0              1.0             20.196789
                 2.0             16.277862
                 3.0             12.061866
                 4.0              0.310638
3.0              1.0             19.839760
                 2.0             16.203684
                 3.0             11.231576
                 4.0              0.022131
4.0              1.0             20.463335
                 2.0             16.687550
                 3.0             11.233845
                 4.0              1.112188
5.0              1.0             19.285617
                 2.0    

In [79]:
df.pivot_table(index=['payment_type', 'VendorID'],
               columns='passenger_count',
               values=['total_amount', 'trip_distance'],
               aggfunc=['mean', 'std'])

mean                                   \
                      total_amount                                    
passenger_count                0.0        1.0        2.0        3.0   
payment_type VendorID                                                 
1.0          1.0         19.042514  18.908395  20.163305  19.715304   
             2.0         41.283312  19.784228  20.212669  19.883109   
2.0          1.0         14.994231  14.523719  16.553966  16.801301   
             2.0         44.495783  15.626498  16.136692  15.953876   
3.0          1.0         14.915132  15.183049  20.399819  19.813648   
             2.0        -61.675000 -12.685527 -13.063912 -12.036065   
4.0          1.0         15.361985  16.519574  19.731170  18.078141   
             2.0        -13.612500 -15.469029 -18.036862 -16.997432   
5.0          1.0               NaN   0.000000        NaN        NaN   

                                                                              \
                                                                               
passenger_count              4.0        5.0        6.0        7.0        8.0   
payment_type VendorID                                                          
1.0          1.0       20.756658  20.858342  23.077220  12.350000  10.300000   
             2.0       20.365903  19.274772  19.380297  80.296364  56.559231   
2.0          1.0       17.253524  18.632354  16.186094  36.420000        NaN   
             2.0       16.358619  15.352934  15.551475  49.572000  81.260000   
3.0          1.0       19.824308  21.691818  32.492800        NaN        NaN   
             2.0      -12.599451 -11.776881  -8.842692        NaN        NaN   
4.0          1.0       22.014338   6.913636   8.549524        NaN        NaN   
             2.0      -22.124213 -11.067987  -7.027843        NaN        NaN   
5.0          1.0             NaN        NaN        NaN        NaN        NaN   

                                  ...           std                      \
                                  ... trip_distance                       
passenger_count              9.0  ...           0.0       1.0       2.0   
payment_type VendorID             ...                                     
1.0          1.0       65.380000  ...      3.635495  3.473871  3.952757   
             2.0       85.738667  ...      1.839421  3.847142  3.963992   
2.0          1.0       61.420000  ...      3.551209  3.257321  4.023848   
             2.0             NaN  ...      3.479518  3.775352  3.955832   
3.0          1.0             NaN  ...      4.393473  4.171511  5.424530   
             2.0             NaN  ...      0.342479  2.908318  1.450897   
4.0          1.0             NaN  ...      5.693314  4.490303  5.392024   
             2.0             NaN  ...      0.000000  3.374683  3.886742   
5.0          1.0             NaN  ...           NaN       NaN       NaN   

                                                                         \
                                                                          
passenger_count             3.0       4.0       5.0       6.0       7.0   
payment_type VendorID                                                     
1.0          1.0       3.813320  4.064551  3.789995  4.515855       NaN   
             2.0       3.872692  4.024464  3.644905  3.759983  7.397031   
2.0          1.0       3.997054  3.979624  4.621283  4.480345       NaN   
             2.0       3.821000  4.373470  3.546435  3.721534  1.390133   
3.0          1.0       4.976179  5.802429  5.871529  5.528629       NaN   
             2.0       1.621301  1.034765  0.325854  0.325101       NaN   
4.0          1.0       4.703409  5.030930  1.134580  3.741377       NaN   
             2.0       4.095468  3.857114  0.287239  0.335431       NaN   
5.0          1.0            NaN       NaN       NaN       NaN       NaN   

                                            
                                            
passenger_count        

In [84]:
df.pivot_table(index='payment_type',
               columns='passenger_count',
               values='total_amount',
               aggfunc='mean',
              margins=True)

total_amount                                              \
passenger_count          0.0        1.0        2.0        3.0        4.0   
payment_type                                                               
1.0                19.169661  19.479882  20.196789  19.839760  20.463335   
2.0                15.080724  15.272954  16.277862  16.203684  16.687550   
3.0                14.753550   9.479660  12.061866  11.231576  11.233845   
4.0                15.009711   0.501119   0.310638   0.022131   1.112188   
5.0                      NaN   0.000000        NaN        NaN        NaN   
All                18.059724  18.343110  19.050504  18.736862  19.128092   

                                                                        \
passenger_count        5.0        6.0        7.0        8.0        9.0   
payment_type                                                             
1.0              19.285617  19.406970  77.342174  53.255000  82.345556   
2.0              15.385476  15.556227  47.380000  81.260000  61.420000   
3.0              -8.708917  -3.133370        NaN        NaN        NaN   
4.0              -8.882376  -2.484444        NaN        NaN        NaN   
5.0                    NaN        NaN        NaN        NaN        NaN   
All              18.234443  18.367962  71.143103  58.197059  81.244211   

                            
passenger_count        All  
payment_type                
1.0              19.602178  
2.0              15.516222  
3.0               9.933257  
4.0               0.890626  
5.0               0.000000  
All              18.471623

# Exercise: Pivot tables with Olympic data

1. Create a pivot table for gold medalists showing mean height for every team vs. sport.
2. Create a pivot table showing mean age and weight for every year vs. team since 2000.

In [86]:
df = pd.read_csv('/Users/reuven/Courses/Current/Data/olympic_athlete_events.csv', 
                 low_memory=False,
                usecols=['Age', 'Height', 'Weight', 'Team', 'Year', 'Sport', 'Medal'])
df

,Age,Height,Weight,Team,Year,Sport,Medal
0,24.0,180.0,80.0,China,1992,Basketball,NaN
1,23.0,170.0,60.0,China,2012,Judo,NaN
2,24.0,NaN,NaN,Denmark,1920,Football,NaN
3,34.0,NaN,NaN,Denmark/Sweden,1900,Tug-Of-War,Gold
4,21.0,185.0,82.0,Netherlands,1988,Speed Skating,NaN
...,...,...,...,...,...,...,...
271111,29.0,179.0,89.0,Poland-1,1976,Luge,NaN
271112,27.0,176.0,59.0,Poland,2014,Ski Jumping,NaN
271113,27.0,176.0,59.0,Poland,2014,Ski Jumping,NaN
271114,30.0,185.0,96.0,Poland,1998,Bobsleigh,NaN


In [91]:
# Create a pivot table for gold medalists showing mean height for every team vs. sport.

# index - 'Team'
# columns - 'Sport'
# values - 'Height'
# aggfunc - 'mean'

(
    df
    .loc[df['Medal'] == 'Gold']
    .pivot_table(index='Team',
               columns='Sport',
               values='Height',
               aggfunc='mean')
    .dropna(thresh=20)
)


Sport,Alpine Skiing,Archery,Art Competitions,Athletics,Badminton,Baseball,Basketball,Beach Volleyball,Biathlon,Bobsleigh,...,Table Tennis,Taekwondo,Tennis,Trampolining,Triathlon,Tug-Of-War,Volleyball,Water Polo,Weightlifting,Wrestling
Team,,,,,,,,,,,,,,,,,,,,,
Australia,NaN,175.0,NaN,172.263158,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,165.000000,180.5000,NaN,161.0,NaN,NaN,178.153846,180.000000,NaN
Canada,166.000000,NaN,NaN,177.380952,NaN,NaN,NaN,NaN,161.000000,NaN,...,NaN,NaN,187.0000,158.0,177.0,NaN,NaN,NaN,NaN,165.666667
China,NaN,169.0,NaN,169.375000,176.375,NaN,NaN,NaN,NaN,NaN,...,171.193548,179.428571,NaN,166.0,NaN,NaN,184.088235,NaN,162.294118,173.500000
East Germany,NaN,NaN,NaN,175.640000,NaN,NaN,NaN,NaN,178.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166.000000,181.000000
France,175.500000,170.0,NaN,176.615385,NaN,NaN,NaN,NaN,174.625000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.666667,172.000000
Germany,174.545455,NaN,NaN,182.760000,NaN,NaN,NaN,NaN,175.588235,NaN,...,NaN,NaN,192.0000,158.0,194.0,179.6,NaN,173.000000,183.000000,176.000000
Great Britain,NaN,NaN,NaN,176.355932,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,156.000000,186.0000,NaN,184.0,NaN,NaN,181.307692,188.000000,175.000000
Italy,172.583333,179.5,NaN,178.055556,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,183.000000,NaN,NaN,NaN,NaN,NaN,180.085714,173.000000,163.800000
Norway,180.666667,NaN,NaN,183.000000,NaN,NaN,NaN,NaN,179.291667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165.000000,162.333333


In [93]:
# Create a pivot table showing mean age and weight for every year vs. team since 2000.

# index -- Team
# columns -- year
# values -- Age and Weight
# aggfunc -- mean

(
    df
    .loc[df['Year'] >= 2000]
    .pivot_table(index='Team',
                 columns='Year',
                 values=['Age', 'Weight'],
                 aggfunc='mean')
)

Age                                                \
Year                 2000  2002       2004       2006       2008   2010   
Team                                                                      
Afghanistan           NaN   NaN  18.600000        NaN  22.500000    NaN   
Albania         31.200000   NaN  20.857143  19.000000  27.250000  23.00   
Algeria         24.901961   NaN  25.084507  24.333333  25.210526  17.00   
American Samoa  27.000000   NaN  30.000000        NaN  23.500000    NaN   
Andorra         31.000000  24.6  29.666667  24.800000  26.600000  23.55   
...                   ...   ...        ...        ...        ...    ...   
Whisper               NaN   NaN        NaN        NaN  67.000000    NaN   
Whitini Star          NaN   NaN        NaN        NaN  36.000000    NaN   
Yemen           25.000000   NaN  20.000000        NaN  21.375000    NaN   
Zambia          23.000000   NaN  22.500000        NaN  21.875000    NaN   
Zimbabwe        24.961538   NaN  25.071429        NaN  26.062500    NaN   

                                                Weight                   \
Year                 2012  2014       2016        2000  2002       2004   
Team                                                                      
Afghanistan     24.833333   NaN  24.666667         NaN   NaN  64.750000   
Albania         25.700000  20.0  23.666667   62.900000   NaN  70.714286   
Algeria         24.846154   NaN  23.959459   67.941176   NaN  67.594203   
American Samoa  22.000000   NaN  25.250000  103.000000   NaN  91.666667   
Andorra         32.000000  23.5  26.000000   69.800000  75.0  68.500000   
...                   ...   ...        ...         ...   ...        ...   
Whisper               NaN   NaN        NaN         NaN   NaN        NaN   
Whitini Star          NaN   NaN        NaN         NaN   NaN        NaN   
Yemen           20.000000   NaN  19.333333   61.000000   NaN  64.333333   
Zambia          22.571429   NaN  24.142857   64.833333   NaN  64.500000   
Zimbabwe        27.333333  20.0  27.483871   71.080000   NaN  70.500000   

                                                                          
Year                 2006       2008   2010       2012   2014       2016  
Team                                                                      
Afghanistan           NaN  62.750000    NaN  60.833333    NaN  74.000000  
Albania         74.000000  74.750000  74.00  80.200000  56.00  67.166667  
Algeria         62.666667  70.821429  65.00  66.857143    NaN  68.378378  
American Samoa        NaN  59.000000    NaN  79.750000    NaN  75.250000  
Andorra         78.100000  61.400000  71.45  68.000000  67.75  66.250000  
...                   ...        ...    ...        ...    ...        ...  
Whisper               NaN  62.000000    NaN        NaN    NaN        NaN  
Whitini Star          NaN        NaN    NaN        NaN    NaN        NaN  
Yemen                 NaN  55.571429    NaN  58.000000    NaN  65.666667  
Zambia                NaN  62.750000    NaN  74.166667    NaN  67.500000  
Zimbabwe              NaN  63.714286    NaN  65.222222  81.00  62.193548  

[331 rows x 18 columns]

# stack + unstack

We saw that a two-dimensional `groupby` and a pivot table are basically the same, just displayed differently. How can we move from one depiction to the other?

The answer is `stack` and `unstack`, two methods that are for precisely this purpose.

- `stack` means: Take the column labels, and move them into the index, such that we have a multi-index on the rows.
- `unstack` means: Take one of the tiers of the multi-index on the rows of a series, and create a data frame where that tier becomes the column names.

In [100]:
# given a multi-indexed series, we can take one level of the index and use it as the 
# columns in a data frame. That's known as "unstack".

df.groupby(['Team', 'Year'])['Age'].mean().unstack('Year')

Year,1896,1900,1904,1906,1908,1912,1920,1924,1928,1932,...,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016
Team,,,,,,,,,,,,,,,,,,,,,
30. Februar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A North American Team,NaN,41.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acipactli,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acturus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,18.600000,NaN,22.5000,NaN,24.833333,NaN,24.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,23.000000,NaN,22.500000,NaN,21.8750,NaN,22.571429,NaN,24.142857
Zefyros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,...,NaN,24.961538,NaN,25.071429,NaN,26.0625,NaN,27.333333,20.0,27.483871


In [101]:
df.groupby(['Team', 'Year'])['Age'].mean().unstack('Year').stack()

Team                   Year
30. Februar            1952    33.500000
A North American Team  1900    41.333333
Acipactli              1964    47.333333
Acturus                1948    27.000000
Afghanistan            1936    24.266667
                                 ...    
Zimbabwe               2012    27.333333
                       2014    20.000000
                       2016    27.483871
Zut                    1908    32.000000
rn-2                   1912    29.200000
Length: 5061, dtype: float64

# Next up: Joining and merging

Resume at :55

In [102]:
from pandas import Series, DataFrame

In [103]:
import numpy as np

In [108]:
np.random.seed(0)

df1 = DataFrame(np.random.randint(0, 1000, [3, 4]),
                index=list('abc'),
                columns=list('wxyz'))

df2 = DataFrame(np.random.randint(0, 1000, [3, 4]),
                index=list('abc'),
                columns=list('wxyz'))

df3 = DataFrame(np.random.randint(0, 1000, [3, 4]),
                index=list('abc'),
                columns=list('uvwx'))


In [109]:
df1

,w,x,y,z
a,684,559,629,192
b,835,763,707,359
c,9,723,277,754


In [110]:
df2

,w,x,y,z
a,804,599,70,472
b,600,396,314,705
c,486,551,87,174


In [111]:
# how can I combine these into a single data frame, keeping all of the original rows and columns?

# Option 1: stack them on top of one another
# Option 2: stack them side-by-side

# we can do this with pd.concat, which takes a list of data frames and returns a new data frame combining them

pd.concat([df1, df2])

,w,x,y,z
a,684,559,629,192
b,835,763,707,359
c,9,723,277,754
a,804,599,70,472
b,600,396,314,705
c,486,551,87,174


In [112]:
pd.concat([df1, df3])

,w,x,y,z,u,v
a,684,559,629.0,192.0,NaN,NaN
b,835,763,707.0,359.0,NaN,NaN
c,9,723,277.0,754.0,NaN,NaN
a,677,537,NaN,NaN,600.0,849.0
b,777,916,NaN,NaN,845.0,72.0
c,755,709,NaN,NaN,115.0,976.0


In [113]:
pd.concat([df1, df2, df3])

,w,x,y,z,u,v
a,684,559,629.0,192.0,NaN,NaN
b,835,763,707.0,359.0,NaN,NaN
c,9,723,277.0,754.0,NaN,NaN
a,804,599,70.0,472.0,NaN,NaN
b,600,396,314.0,705.0,NaN,NaN
c,486,551,87.0,174.0,NaN,NaN
a,677,537,NaN,NaN,600.0,849.0
b,777,916,NaN,NaN,845.0,72.0
c,755,709,NaN,NaN,115.0,976.0


In [115]:
pd.concat([df1, df3[['u', 'v']]])

,w,x,y,z,u,v
a,684.0,559.0,629.0,192.0,NaN,NaN
b,835.0,763.0,707.0,359.0,NaN,NaN
c,9.0,723.0,277.0,754.0,NaN,NaN
a,NaN,NaN,NaN,NaN,600.0,849.0
b,NaN,NaN,NaN,NaN,845.0,72.0
c,NaN,NaN,NaN,NaN,115.0,976.0


In [116]:
# what if we want to join them side-by-side?
# we can pass axis='columns'

pd.concat([df1, df2], axis='columns')

,w,x,y,z,w,x,y,z
a,684,559,629,192,804,599,70,472
b,835,763,707,359,600,396,314,705
c,9,723,277,754,486,551,87,174


# When do I use `pd.concat`?

Most often: When I have data split across multiple files. I can read the files into a list of data frames, and then use `pd.concat` to combine them into a single data frame.

The big thing to be sure of is that if you're combining them top-to-bottom, that the columns (or most of the columns) match up.

In [119]:
pd.concat([df1, df2]).reset_index()

,index,w,x,y,z
0,a,684,559,629,192
1,b,835,763,707,359
2,c,9,723,277,754
3,a,804,599,70,472
4,b,600,396,314,705
5,c,486,551,87,174


In [120]:
pd.concat([df1, df2], ignore_index=True)

,w,x,y,z
0,684,559,629,192
1,835,763,707,359
2,9,723,277,754
3,804,599,70,472
4,600,396,314,705
5,486,551,87,174


In [117]:
help(pd.concat)

Help on function concat in module pandas.core.reshape.concat:

concat(
    objs: 'Iterable[Series | DataFrame] | Mapping[HashableT, Series | DataFrame]',
    *,
    axis: 'Axis' = 0,
    join: 'str' = 'outer',
    ignore_index: 'bool' = False,
    keys: 'Iterable[Hashable] | None' = None,
    levels=None,
    names: 'list[HashableT] | None' = None,
    verify_integrity: 'bool' = False,
    sort: 'bool' = False,
    copy: 'bool | None' = None
) -> 'DataFrame | Series'
    Concatenate pandas objects along a particular axis.

    Allows optional set logic along the other axes.

    Can also add a layer of hierarchical indexing on the concatenation axis,
    which may be useful if the labels are the same (or overlapping) on
    the passed axis number.

    Parameters
    ----------
    objs : a sequence or mapping of Series or DataFrame objects
        If a mapping is passed, the sorted keys will be used as the `keys`
        argument, unless it is passed, in which case the values will be


# Exercise: Concatenation and analysis

We just used the taxi information from January, 2020 in New York. There are actually *four* files of taxi information -- from January 2020 and 2021, and July 2020 and 2021.

1. Load all four of these into a single data frame using `pd.concat`. If this is too much for your computer, then you can load two of them -- the two files from 2020 are probably the best bets.
2. Find the mean and median `trip_distance` and `total_amount` for each `passenger_count`.
3. Find the number of trips in which people were refunded money. How far, on average, did such people travel?
4. Find the number of trips in which people went 0 miles. How much did they, on average, pay for the privilege?


In [122]:
!ls /Users/reuven/Courses/Current/Data/nyc_taxi_*.csv

/Users/reuven/Courses/Current/Data/nyc_taxi_2019-01.csv
/Users/reuven/Courses/Current/Data/nyc_taxi_2019-07.csv
/Users/reuven/Courses/Current/Data/nyc_taxi_2020-01.csv
/Users/reuven/Courses/Current/Data/nyc_taxi_2020-07.csv


In [123]:
import glob  

glob.glob('/Users/reuven/Courses/Current/Data/nyc_taxi_*.csv')

['/Users/reuven/Courses/Current/Data/nyc_taxi_2020-01.csv',
 '/Users/reuven/Courses/Current/Data/nyc_taxi_2020-07.csv',
 '/Users/reuven/Courses/Current/Data/nyc_taxi_2019-07.csv',
 '/Users/reuven/Courses/Current/Data/nyc_taxi_2019-01.csv']

In [125]:
all_dfs = []

for one_filename in glob.glob('/Users/reuven/Courses/Current/Data/nyc_taxi_*.csv'):
    print(one_filename)
    all_dfs.append(pd.read_csv(one_filename, usecols=['trip_distance', 'total_amount', 'passenger_count']))

len(all_dfs)

/Users/reuven/Courses/Current/Data/nyc_taxi_2020-01.csv
/Users/reuven/Courses/Current/Data/nyc_taxi_2020-07.csv
/Users/reuven/Courses/Current/Data/nyc_taxi_2019-07.csv
/Users/reuven/Courses/Current/Data/nyc_taxi_2019-01.csv


4

In [126]:
df = pd.concat(all_dfs)  
df.shape

(21183631, 3)

In [128]:
# I like to use list comprehensions!

df = pd.concat([pd.read_csv(one_filename, 
                            usecols=['trip_distance', 'total_amount', 'passenger_count'])
                for one_filename in glob.glob('/Users/reuven/Courses/Current/Data/nyc_taxi_*.csv')])
df.shape

(21183631, 3)

In [129]:
# Find the mean and median trip_distance and total_amount for each passenger_count.

df.groupby('passenger_count')[['trip_distance', 'total_amount']].agg(['mean', 'median'])

trip_distance        total_amount       
                         mean median         mean median
passenger_count                                         
0.0                  2.739365   1.50    18.464536  13.30
1.0                  2.851220   1.60    17.565138  13.30
2.0                  3.029851   1.64    18.162595  13.56
3.0                  3.009260   1.64    18.031471  13.56
4.0                  3.101422   1.70    18.573759  13.80
5.0                  2.938570   1.63    17.509767  13.39
6.0                  2.949152   1.62    17.453694  13.30
7.0                  3.845976   0.01    62.500732  75.80
8.0                  3.219714   0.00    64.482000  84.15
9.0                  5.229778   0.00    74.031111  92.80

In [133]:
# Find the number of trips in which people were refunded money. How far, on average, did such people travel?

(
    df.loc[df['total_amount'] < 0,   # row selector
           'trip_distance']          # column selector
    .describe()
)

count    43620.000000
mean         1.511473
std          3.254798
min         -4.410000
25%          0.050000
50%          0.530000
75%          1.100000
max         91.410000
Name: trip_distance, dtype: float64

In [135]:
# Find the number of trips in which people went 0 miles. How much did they, on average, pay for the privilege?

(
    df.loc[df['trip_distance'] == 0,    # row selector
          'total_amount']               # column selector
    .mean()
)

np.float64(33.38460416115051)

# Joins and merges

If we have two data frames with related/overlapping data, we can `join` or `merge` them together. This is the same thing as joins in SQL. The big difference between `join` and `merge` is that `join` only works on the indexes, joining the two data frames together on their indexes. `merge`, by contrast, can be used with any two columns.



In [136]:
people = DataFrame({'age': [54, 23, 21, 19],
                    'shoesize': [46, 40, 40, 44]},
                   index=['Reuven', 'Atara', 'Shikma', 'Amotz'])
people

,age,shoesize
Reuven,54,46
Atara,23,40
Shikma,21,40
Amotz,19,44


In [138]:
gender = DataFrame({'g': ['m', 'f', 'f', 'm']},
                   index=['Reuven', 'Atara', 'Shikma', 'Amotz'])
gender

,g
Reuven,m
Atara,f
Shikma,f
Amotz,m


In [139]:
pd.concat([people, gender], axis='columns')

,age,shoesize,g
Reuven,54,46,m
Atara,23,40,f
Shikma,21,40,f
Amotz,19,44,m


In [141]:
people.join(gender)   # we start with people, and then look, for each index on people, for a corresponding row on gender

,age,shoesize,g
Reuven,54,46,m
Atara,23,40,f
Shikma,21,40,f
Amotz,19,44,m


In [151]:
products = DataFrame({'name': ['apple', 'banana', 'celery', 'dill', 'pet rock'],
                      'price': [5, 3, 2, 1, 3],
                      'dept': ['fruit', 'fruit', 'veg', 'veg', 'dumb toys']})
products

,name,price,dept
0,apple,5,fruit
1,banana,3,fruit
2,celery,2,veg
3,dill,1,veg
4,pet rock,3,dumb toys


In [152]:
# this data frame will record each time someone bought one of our products
# it'll have the product name and the quantity that was bought

sales = DataFrame({'name': ['apple', 'apple', 'apple', 'banana', 'celery', 'dill', 'dill', 'celery', 'banana'],
                   'quantity': [1, 2, 3, 1, 2, 3, 1, 2, 3]})
sales

,name,quantity
0,apple,1
1,apple,2
2,apple,3
3,banana,1
4,celery,2
5,dill,3
6,dill,1
7,celery,2
8,banana,3


In [145]:
# how many did we sell of each product?
# how much did we receive in revenue?
# how much did each product earn us?

In [155]:
# if I want, I can *merge* these, because the indexes don't match up

products_and_sales = pd.merge(left=products, right=sales,
        on='name')
products_and_sales

,name,price,dept,quantity
0,apple,5,fruit,1
1,apple,5,fruit,2
2,apple,5,fruit,3
3,banana,3,fruit,1
4,banana,3,fruit,3
5,celery,2,veg,2
6,celery,2,veg,2
7,dill,1,veg,3
8,dill,1,veg,1


In [156]:
# how many did we sell of each product?

products_and_sales['quantity'].sum()

np.int64(18)

In [159]:
# how much did we receive in revenue?

(products_and_sales['price'] * products_and_sales['quantity']).sum()

np.int64(54)

In [160]:
# how much did each product earn us?

products_and_sales['revenue'] = (products_and_sales['price'] * products_and_sales['quantity'])

products_and_sales

,name,price,dept,quantity,revenue
0,apple,5,fruit,1,5
1,apple,5,fruit,2,10
2,apple,5,fruit,3,15
3,banana,3,fruit,1,3
4,banana,3,fruit,3,9
5,celery,2,veg,2,4
6,celery,2,veg,2,4
7,dill,1,veg,3,3
8,dill,1,veg,1,1


In [162]:
products_and_sales.groupby('name')['revenue'].sum()

name
apple     30
banana    12
celery     8
dill       4
Name: revenue, dtype: int64

In [163]:
products_and_sales.groupby('dept')['revenue'].sum()

dept
fruit    42
veg      12
Name: revenue, dtype: int64

In [164]:
products = products.set_index('name')
sales = sales.set_index('name')

products.join(sales)   # the index of our result depends on the left-side data frame (i.e., the df we're runinng .join on)

,price,dept,quantity
name,,,
apple,5,fruit,1.0
apple,5,fruit,2.0
apple,5,fruit,3.0
banana,3,fruit,1.0
banana,3,fruit,3.0
celery,2,veg,2.0
celery,2,veg,2.0
dill,1,veg,3.0
dill,1,veg,1.0


In [165]:
# if we turn it around, we won't see "pet rock"

sales.join(products)

,quantity,price,dept
name,,,
apple,1,5,fruit
apple,2,5,fruit
apple,3,5,fruit
banana,1,3,fruit
celery,2,2,veg
dill,3,1,veg
dill,1,1,veg
celery,2,2,veg
banana,3,3,fruit


In [166]:
# what if I really want to get everything in products, even when I primarily join on sales?
# in other words, we want the "right side" data frame to drive the index
# that's known as a "right join"

sales.join(products, how='right')

,quantity,price,dept
name,,,
apple,1.0,5,fruit
apple,2.0,5,fruit
apple,3.0,5,fruit
banana,1.0,3,fruit
banana,3.0,3,fruit
celery,2.0,2,veg
celery,2.0,2,veg
dill,3.0,1,veg
dill,1.0,1,veg


In [167]:
sales.join(products, how='right').groupby('name')['price'].sum()

name
apple       15
banana       6
celery       4
dill         2
pet rock     3
Name: price, dtype: int64

In [168]:
products

,price,dept
name,,
apple,5,fruit
banana,3,fruit
celery,2,veg
dill,1,veg
pet rock,3,dumb toys


In [169]:
sales

,quantity
name,
apple,1
apple,2
apple,3
banana,1
celery,2
dill,3
dill,1
celery,2
banana,3


In [170]:
products['x'] = [10, 20, 30, 40, 50]
sales['x'] = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [171]:
products.join(sales)

ValueError: columns overlap but no suffix specified: Index(['x'], dtype='object')

In [173]:
products.join(sales, lsuffix='_products', rsuffix='_sales')

,price,dept,x_products,quantity,x_sales
name,,,,,
apple,5,fruit,10,1.0,10.0
apple,5,fruit,10,2.0,20.0
apple,5,fruit,10,3.0,30.0
banana,3,fruit,20,1.0,40.0
banana,3,fruit,20,3.0,90.0
celery,2,veg,30,2.0,50.0
celery,2,veg,30,2.0,80.0
dill,1,veg,40,3.0,60.0
dill,1,veg,40,1.0,70.0


In [174]:
glob.glob('/Users/reuven/Courses/Current/Data/oecd*')

['/Users/reuven/Courses/Current/Data/oecd_tourism.csv',
 '/Users/reuven/Courses/Current/Data/oecd_locations.csv']

In [175]:
!head /Users/reuven/Courses/Current/Data/oecd_locations.csv

﻿AUS,Australia
AUT,Austria
BEL,Belgium
CAN,Canada
DNK,Denmark
FIN,Finland
FRA,France
DEU,Germany
HUN,Hungary
ITA,Italy


In [176]:
!head /Users/reuven/Courses/Current/Data/oecd_tourism.csv

﻿"LOCATION","INDICATOR","SUBJECT","MEASURE","FREQUENCY","TIME","Value","Flag Codes"
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2008",31159.8,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2009",29980.7,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2010",35165.5,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2011",38710.1,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2012",38003.7,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2013",36965,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2014",38047.9,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2015",36226,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2016",39082.3,


In [177]:
!tail /Users/reuven/Courses/Current/Data/oecd_tourism.csv

"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2010",1102.353,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2011",1263.55,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2012",1187.555,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2013",1289.398,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2014",1359.859,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2015",1253.644,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2016",1351.098,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2017",1549.183,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2018",1837.317,
"SRB","TOUR_REC_EXP","INT-EXP","USD","A","2019",1999.313,


# Exercise: OECD tourism

1. Create data frames from both the OECD tourism data and from the OECD locations data.
2. Join them together, such that we can find out which 5 countries spent (`INT-EXP`) the most money annual (`A`) in 2019. We want to know the name of the countries, not their abbreviations.
3. Similarly, which 5 countries received (`INT_REC`) the least amount of money, on average, over all of the years. Show the names, not abbreviations.

In [188]:
locations_df = pd.read_csv('/Users/reuven/Courses/Current/Data/oecd_locations.csv',
                          header=None,
                          names=['LOCATION', 'NAME'],
                          index_col='LOCATION')
tourism_df = pd.read_csv('/Users/reuven/Courses/Current/Data/oecd_tourism.csv',
                        index_col='LOCATION')


In [189]:
locations_df.head()

,NAME
LOCATION,
AUS,Australia
AUT,Austria
BEL,Belgium
CAN,Canada
DNK,Denmark


In [190]:
tourism_df.head()

,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
LOCATION,,,,,,,
AUS,TOUR_REC_EXP,INT_REC,USD,A,2008,31159.8,NaN
AUS,TOUR_REC_EXP,INT_REC,USD,A,2009,29980.7,NaN
AUS,TOUR_REC_EXP,INT_REC,USD,A,2010,35165.5,NaN
AUS,TOUR_REC_EXP,INT_REC,USD,A,2011,38710.1,NaN
AUS,TOUR_REC_EXP,INT_REC,USD,A,2012,38003.7,NaN


In [193]:
df = locations_df.join(tourism_df)
df.head(20)

,NAME,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
LOCATION,,,,,,,,
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2008,31159.8,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2009,29980.7,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2010,35165.5,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2011,38710.1,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2012,38003.7,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2013,36965.0,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2014,38047.9,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2015,36226.0,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2016,39082.3,NaN


In [200]:
# Join them together, such that we can find out which 5 countries spent (INT-EXP) the most money annual (A) in 2019. 
# We want to know the name of the countries, not their abbreviations.

(
    df
    .loc[(df['SUBJECT'] == 'INT-EXP')  &  (df['TIME'] == 2019)]
    .sort_values('Value', ascending=False)
    .head(5)  # get the top 5
    [['NAME', 'Value']]
)

,NAME,Value
LOCATION,,
USA,United States,182365.000
CAN,Canada,43681.500
AUS,Australia,41452.000
JPN,Japan,29130.000
BRA,Brazil,21177.995


In [202]:
# Similarly, which 5 countries received (INT_REC) the least amount of money, on average, over all of the years. 
# Show the names, not abbreviations.

(
    df
    .loc[df['SUBJECT'] == 'INT_REC']   # keep only INT_REC lines
    .groupby('NAME')['Value'].mean()   # for each country name, find the mean value
)

NAME
Australia          38540.900000
Austria            21413.209091
Belgium            12190.591636
Brazil              6321.476083
Canada             24202.591667
Denmark             9398.957636
Finland             4700.236273
France             65063.335727
Germany            53408.570636
Hungary             7299.353000
Israel              6542.383250
Italy              44930.211545
Japan              25015.858333
Korea              17780.754545
United Kingdom     51752.090909
United States     201613.500000
Name: Value, dtype: float64

# Next up

1. Method chaining -- more sophistication
    - `lambda`
    - `assign`
    - `pipe`
2. Working with text in our Pandas data frames

Resume at 13:20 Paris Time

# Method chaining

The basic idea is pretty clear:

- We call a method on a data frame, and get back a new data frame
- Because this new data frame is (obviously) a data frame, we can invoke other data-frame methods on it
- We keep calling a method on the latest data frame we got back
- ... until we get a solution

In [205]:
# in this example, I load OECD locations into a data frame, I then join that
# with the OECD tourism data, then I filter by INT_REC, then I groupby NAME / Value

(
    pd
    .read_csv('/Users/reuven/Courses/Current/Data/oecd_locations.csv',
               header=None,
               names=['LOCATION', 'NAME'],
               index_col='LOCATION')
    .join(pd
          .read_csv('/Users/reuven/Courses/Current/Data/oecd_tourism.csv',
                        index_col='LOCATION')
         )
    .loc[  df['SUBJECT'] == 'INT_REC'  ]   # keep only INT_REC lines
    .groupby('NAME')['Value'].mean()       
)


NAME
Australia          38540.900000
Austria            21413.209091
Belgium            12190.591636
Brazil              6321.476083
Canada             24202.591667
Denmark             9398.957636
Finland             4700.236273
France             65063.335727
Germany            53408.570636
Hungary             7299.353000
Israel              6542.383250
Italy              44930.211545
Japan              25015.858333
Korea              17780.754545
United Kingdom     51752.090909
United States     201613.500000
Name: Value, dtype: float64

# `.loc`

We know that `.loc` can be invoked in a number of different ways:

- On a series, specifying which single index we want
- On a series, specifying more than one index (fancy indexing)
- On a series, with a boolean series, as a mask index

- On a data frame, specifying which single index (row) we want
- On a data frame, specifying more than one index (fancy indexing) for multiple rows
- On a data frame, with a boolean series, as a mask index, to get multiple rows

- On a data frame, with any of the above arguments in the first location and then a column selector in the second location
- (column selector can be a single column or a list of columns)

- Another option, for both series and data frames: We provide a function that is invoked on the series or the data frame
- It returns a boolean series or a boolean data frame. This then acts as a mask index


In [208]:
# I want all of the rows in which `Name` is `Australia`

def is_australia(df_):
    return df_['NAME'] == 'Australia'

df.loc[is_australia]   # I pass the function as an argument to df.loc. It is invoked *by* df.loc on df

,NAME,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
LOCATION,,,,,,,,
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2008,31159.8,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2009,29980.7,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2010,35165.5,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2011,38710.1,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2012,38003.7,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2013,36965.0,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2014,38047.9,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2015,36226.0,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2016,39082.3,NaN


In [209]:
# I want all of the rows in which `Name` is `Australia` and the year is > 2015

def is_australia(df_):
    return (df_['NAME'] == 'Australia') & (df['TIME'] > 2015)

df.loc[is_australia]   # I pass the function as an argument to df.loc. It is invoked *by* df.loc on df

,NAME,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
LOCATION,,,,,,,,
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2016,39082.3,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2017,43959.1,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2018,47259.8,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2019,47930.9,NaN
AUS,Australia,TOUR_REC_EXP,INT-EXP,USD,A,2016,35674.6,NaN
AUS,Australia,TOUR_REC_EXP,INT-EXP,USD,A,2017,39644.9,NaN
AUS,Australia,TOUR_REC_EXP,INT-EXP,USD,A,2018,42478.0,NaN
AUS,Australia,TOUR_REC_EXP,INT-EXP,USD,A,2019,41452.0,NaN


In [210]:
# we can accomplish the same thing by writing an inline, anonymous function rather than
# writing a function, giving it a name, and passing it.

# lambda is Python's way to create anonymous, inline functions
# when we use lambda, we create a function but we don't give it a name

In [211]:
def square(x):
    return x ** 2

square(3)

9

# What happens when we define a function?

1. We create a function object (compiled into bytecode, with lots of hints, and then turned into a "code" object)
2. That function object is assigned to the name we gave

Functions aren't in their own namespace. They occupy the same namespace as all other variables in Python.

In [212]:
type(square)

function

In [213]:
lambda x: x**2

<function __main__.<lambda>(x)>

In [215]:
# this is basically the same as the def I did before

square = lambda x: x**2

In [216]:
square(3)

9

# `lambda` syntax and restrictions

A `lambda` statement must be on a single line of code. (More, if you have parentheses.) It contains a single expression, and that expression is returned to the caller.

You cannot have any statements in your `lambda` body:

- No `if`/`else`
- No assignment
- No `for` loops
- No `return` statement

Now, we can:
- Use the trinary version of `if`/`else` (which I dislike)
- Use comprehensions
- The value of the expression is returned to the caller

Syntax of `lambda` is:
- `lambda`
- one or more parameters that the function will take, traditionally without parentheses, with commas between them
- then a `:`
- the single-line expression that forms the function body

In [218]:

# def is_australia(df_):
#     return (df_['NAME'] == 'Australia') & (df['TIME'] > 2015)

(
    df
    .loc[lambda df_: df_['NAME'] == 'Australia']   # here, our lambda gets df as an argument
    .loc[lambda df_: df_['TIME'] > 2015]           # here, our lambda gets the output from the 1st lambda as an argument
)

,NAME,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
LOCATION,,,,,,,,
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2016,39082.3,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2017,43959.1,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2018,47259.8,NaN
AUS,Australia,TOUR_REC_EXP,INT_REC,USD,A,2019,47930.9,NaN
AUS,Australia,TOUR_REC_EXP,INT-EXP,USD,A,2016,35674.6,NaN
AUS,Australia,TOUR_REC_EXP,INT-EXP,USD,A,2017,39644.9,NaN
AUS,Australia,TOUR_REC_EXP,INT-EXP,USD,A,2018,42478.0,NaN
AUS,Australia,TOUR_REC_EXP,INT-EXP,USD,A,2019,41452.0,NaN


In [219]:
# use lambda to ensure that we don't confuse "df" with the data frame on which we're really working

(
    pd
    .read_csv('/Users/reuven/Courses/Current/Data/oecd_locations.csv',
               header=None,
               names=['LOCATION', 'NAME'],
               index_col='LOCATION')
    .join(pd
          .read_csv('/Users/reuven/Courses/Current/Data/oecd_tourism.csv',
                        index_col='LOCATION')
         )

    # here, we're using a lambda expression with df_, which means its input is whatever the join returned
    .loc[ lambda df_: df_['SUBJECT'] == 'INT_REC'  ]   
    .groupby('NAME')['Value'].mean()       
)


NAME
Australia          38540.900000
Austria            21413.209091
Belgium            12190.591636
Brazil              6321.476083
Canada             24202.591667
Denmark             9398.957636
Finland             4700.236273
France             65063.335727
Germany            53408.570636
Hungary             7299.353000
Israel              6542.383250
Italy              44930.211545
Japan              25015.858333
Korea              17780.754545
United Kingdom     51752.090909
United States     201613.500000
Name: Value, dtype: float64

In [220]:
# let's keep only those values that are > 10,000

(
    pd
    .read_csv('/Users/reuven/Courses/Current/Data/oecd_locations.csv',
               header=None,
               names=['LOCATION', 'NAME'],
               index_col='LOCATION')
    .join(pd
          .read_csv('/Users/reuven/Courses/Current/Data/oecd_tourism.csv',
                        index_col='LOCATION')
         )

    # here, we're using a lambda expression with df_, which means its input is whatever the join returned
    .loc[ lambda df_: df_['SUBJECT'] == 'INT_REC'  ]   
    .groupby('NAME')['Value'].mean()    

    # let's use a lambda on our series and keep only the value > 10k
    .loc[ lambda s_: s_ > 10_000 ]
)


NAME
Australia          38540.900000
Austria            21413.209091
Belgium            12190.591636
Canada             24202.591667
France             65063.335727
Germany            53408.570636
Italy              44930.211545
Japan              25015.858333
Korea              17780.754545
United Kingdom     51752.090909
United States     201613.500000
Name: Value, dtype: float64

# Exercise: Using `.loc` and `lambda` 

1. Load the short (`taxi.csv`) taxi data into a data frame.
2. Find (using `.loc` and `lambda`) all rows in which the number of passengers is 3 and the `total_amount` is > 5.
3. Find (using `.loc` and `lambda`) all rows in which the `trip_distance` is less than the median trip distance but greater than the median `total_amount`.
4. Find all rows in which the number of passengers is either 2 or 3, and the payment type is either 1 or 2.

In [227]:
df = pd.read_csv('/Users/reuven/Courses/Current/Data/nyc_taxi_2020-01.csv', low_memory=False)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [228]:
%%timeit

# 2. Find (using `.loc` and `lambda`) all rows in which the number of passengers is 3 and the `total_amount` is > 5.

(
    df
    .loc[lambda df_: df_['passenger_count'] == 3]   # .loc calls our lambda, with df as an argument, such that df_ ==  (global) df
    .loc[lambda df_: df_['total_amount'] > 5]       # .loc calls our lambda, with the output from line 5 as the argument
)

46.5 ms ± 623 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [229]:
%%timeit

(
    df
    .loc[(df['passenger_count'] == 3)   &   (df['total_amount'] > 5)]
)

36.4 ms ± 571 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [231]:
(
    df
    .loc[df['passenger_count'] == 3]
    .loc[df['total_amount'] > 5]
)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
15,1.0,2020-01-01 00:15:35,2020-01-01 00:27:06,3.0,1.60,1.0,N,211,234,2.0,9.0,3.0,0.5,0.00,0.0,0.3,12.80,2.5
51,2.0,2020-01-01 00:05:31,2020-01-01 00:25:18,3.0,3.36,1.0,N,237,107,1.0,15.0,0.5,0.5,3.76,0.0,0.3,22.56,2.5
63,2.0,2020-01-01 00:20:37,2020-01-01 00:26:18,3.0,1.18,1.0,N,237,239,2.0,6.0,0.5,0.5,0.00,0.0,0.3,9.80,2.5
64,2.0,2020-01-01 00:29:55,2020-01-01 00:48:40,3.0,2.42,1.0,N,239,229,1.0,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5
67,1.0,2020-01-01 00:53:50,2020-01-01 01:04:39,3.0,1.30,1.0,N,107,170,2.0,8.5,3.0,0.5,0.00,0.0,0.3,12.30,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339513,2.0,2020-01-31 23:01:17,2020-01-31 23:08:14,3.0,0.70,1.0,N,148,144,2.0,6.0,0.5,0.5,0.00,0.0,0.3,9.80,2.5
6339516,2.0,2020-01-31 23:54:20,2020-02-01 00:12:24,3.0,2.23,1.0,N,90,79,1.0,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5
6339554,2.0,2020-01-31 22:59:01,2020-01-31 23:06:15,3.0,0.90,1.0,N,238,239,2.0,6.5,0.5,0.5,0.00,0.0,0.3,10.30,2.5
6339555,2.0,2020-01-31 23:19:46,2020-01-31 23:34:34,3.0,2.09,1.0,N,142,141,1.0,10.0,0.5,0.5,3.45,0.0,0.3,17.25,2.5


In [233]:
# 3. Find (using `.loc` and `lambda`) all rows in which the `trip_distance` is less than the median trip distance 
# but greater than the median `total_amount`.

(
    df
    .loc[lambda df_: df_['trip_distance'] < df_['trip_distance'].median()]
    .loc[lambda df_: df_['total_amount'] > df_['total_amount'].median()]
)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.00,3.00,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,7.00,3.00,0.5,1.50,0.0,0.3,12.30,2.5
9,1.0,2020-01-01 00:29:01,2020-01-01 00:40:28,2.0,0.70,1.0,N,246,48,1.0,8.00,3.00,0.5,2.35,0.0,0.3,14.15,2.5
11,1.0,2020-01-01 00:37:15,2020-01-01 00:51:41,1.0,0.80,1.0,N,163,161,2.0,9.50,3.00,0.5,0.00,0.0,0.3,13.30,2.5
13,2.0,2020-01-01 00:21:54,2020-01-01 00:27:31,1.0,1.07,1.0,N,43,239,1.0,6.00,0.50,0.5,1.96,0.0,0.3,11.76,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6404947,NaN,2020-01-31 13:25:22,2020-01-31 13:35:19,NaN,1.45,NaN,NaN,205,122,NaN,28.58,2.75,0.0,0.00,0.0,0.3,31.63,0.0
6404948,NaN,2020-01-31 13:58:35,2020-01-31 14:07:33,NaN,0.98,NaN,NaN,205,205,NaN,28.58,2.75,0.0,0.00,0.0,0.3,31.63,0.0
6404960,NaN,2020-01-31 06:04:00,2020-01-31 06:07:00,NaN,0.15,NaN,NaN,81,51,NaN,30.66,2.75,0.5,0.00,0.0,0.3,34.21,0.0
6404974,NaN,2020-01-31 21:14:39,2020-01-31 21:18:38,NaN,0.41,NaN,NaN,143,143,NaN,13.88,5.50,0.5,0.00,0.0,0.3,20.18,0.0


In [234]:
# 4. Find all rows in which the number of passengers is either 2 or 3, and the payment type is either 1 or 2.

(
    df
    .loc[lambda df_: df_['passenger_count'].isin([2, 3])]
    .loc[lambda df_: df_['payment_type'].isin([1, 2])]
)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
9,1.0,2020-01-01 00:29:01,2020-01-01 00:40:28,2.0,0.70,1.0,N,246,48,1.0,8.0,3.0,0.5,2.35,0.0,0.3,14.15,2.5
10,1.0,2020-01-01 00:55:11,2020-01-01 01:12:03,2.0,2.40,1.0,N,246,79,1.0,12.0,3.0,0.5,1.75,0.0,0.3,17.55,2.5
15,1.0,2020-01-01 00:15:35,2020-01-01 00:27:06,3.0,1.60,1.0,N,211,234,2.0,9.0,3.0,0.5,0.00,0.0,0.3,12.80,2.5
24,1.0,2020-01-01 00:28:02,2020-01-01 00:35:56,2.0,1.50,1.0,N,142,236,1.0,7.5,3.0,0.5,2.80,0.0,0.3,14.10,2.5
28,2.0,2020-01-01 00:31:25,2020-01-01 00:37:16,2.0,0.93,1.0,N,236,262,1.0,5.5,0.5,0.5,2.32,0.0,0.3,11.62,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339539,2.0,2020-01-31 23:26:10,2020-01-31 23:34:01,2.0,0.70,1.0,N,148,79,1.0,6.5,0.5,0.5,1.00,0.0,0.3,11.30,2.5
6339541,1.0,2020-01-31 23:46:39,2020-01-31 23:58:25,2.0,1.20,1.0,N,113,4,2.0,9.0,3.0,0.5,0.00,0.0,0.3,12.80,2.5
6339554,2.0,2020-01-31 22:59:01,2020-01-31 23:06:15,3.0,0.90,1.0,N,238,239,2.0,6.5,0.5,0.5,0.00,0.0,0.3,10.30,2.5
6339555,2.0,2020-01-31 23:19:46,2020-01-31 23:34:34,3.0,2.09,1.0,N,142,141,1.0,10.0,0.5,0.5,3.45,0.0,0.3,17.25,2.5


# What about adding / replacing columns?

One of the things we often want to do when constructing a Pandas query is add a new column (based on existing ones) or replace an existing column's values with something new.

Pandas provides the `assign` method, which takes keyword arguments:
- The name for the keyword argument is the column name that'll be created/replaced
- The value for the keyword argument can be a series or list or `lambda` that is invoked, and whose results are then assigned to the new column

If the name is of a column that already exists, then we replace the existing column.

The columns that we add using `assign` only exist through the query's end. When it's over, the columns are gone -- that's because we have been modifying a chain of temporary data frames.

When we create a dict in modern Python, the key-value pairs are stored in chronological order. Earlier key-value pairs are "first" when we see the dict's contents. This is true also when we pass keyword arguments to a function.  This means that if you pass multiple keyword arguments to `assign`, the latter keyword arguments can use the columns created in the earlier ones.

In [235]:
(
    df
    .assign(paid_with_cc = lambda df_: df_['payment_type'] == 1)
)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,paid_with_cc
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.00,3.00,0.5,1.47,0.00,0.3,11.27,2.5,True
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,7.00,3.00,0.5,1.50,0.00,0.3,12.30,2.5,True
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,N,238,238,1.0,6.00,3.00,0.5,1.00,0.00,0.3,10.80,2.5,True
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1.0,5.50,0.50,0.5,1.36,0.00,0.3,8.16,0.0,True
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,N,193,193,2.0,3.50,0.50,0.5,0.00,0.00,0.3,4.80,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,NaN,2020-01-31 22:51:00,2020-01-31 23:22:00,NaN,3.24,NaN,NaN,237,234,NaN,17.59,2.75,0.5,0.00,0.00,0.3,21.14,0.0,False
6405004,NaN,2020-01-31 22:10:00,2020-01-31 23:26:00,NaN,22.13,NaN,NaN,259,45,NaN,46.67,2.75,0.5,0.00,12.24,0.3,62.46,0.0,False
6405005,NaN,2020-01-31 22:50:07,2020-01-31 23:17:57,NaN,10.51,NaN,NaN,137,169,NaN,48.85,2.75,0.0,0.00,0.00,0.3,51.90,0.0,False
6405006,NaN,2020-01-31 22:25:53,2020-01-31 22:48:32,NaN,5.49,NaN,NaN,50,42,NaN,27.17,2.75,0.0,0.00,0.00,0.3,30.22,0.0,False


In [238]:
(
    df
    .assign(tip_percentage = lambda df_: df_['tip_amount'] / df_['fare_amount'],
           is_big_tipper = lambda df_: df_['tip_percentage'] > 0.2)
    .loc[lambda df_: df_['is_big_tipper']]
)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,tip_percentage,is_big_tipper
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.00,3.0,0.5,1.47,0.00,0.3,11.27,2.5,0.245000,True
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,7.00,3.0,0.5,1.50,0.00,0.3,12.30,2.5,0.214286,True
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1.0,5.50,0.5,0.5,1.36,0.00,0.3,8.16,0.0,0.247273,True
9,1.0,2020-01-01 00:29:01,2020-01-01 00:40:28,2.0,0.70,1.0,N,246,48,1.0,8.00,3.0,0.5,2.35,0.00,0.3,14.15,2.5,0.293750,True
12,1.0,2020-01-01 00:56:27,2020-01-01 01:21:44,1.0,3.30,1.0,N,161,144,1.0,17.00,3.0,0.5,4.15,0.00,0.3,24.95,2.5,0.244118,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6403906,NaN,2020-01-31 01:04:00,2020-01-31 01:35:00,NaN,19.13,NaN,NaN,132,114,NaN,60.00,0.0,0.5,13.88,6.12,0.3,83.30,2.5,0.231333,True
6403989,NaN,2020-01-31 04:15:00,2020-01-31 04:31:00,NaN,6.31,NaN,NaN,249,265,NaN,45.00,0.0,0.0,11.91,11.75,0.3,71.46,2.5,0.264667,True
6404081,NaN,2020-01-31 11:53:00,2020-01-31 11:56:00,NaN,0.46,NaN,NaN,137,224,NaN,65.00,0.0,0.5,13.66,0.00,0.3,81.96,2.5,0.210154,True
6404089,NaN,2020-01-31 11:01:00,2020-01-31 11:41:00,NaN,8.55,NaN,NaN,88,72,NaN,25.71,0.0,0.5,6.87,6.12,0.3,39.50,0.0,0.267211,True


In [248]:
%%timeit

(
    df                                             # 6m rows
    .loc[lambda df_: df_['passenger_count'] > 6]   # after filtering, 65 rows
    .loc[lambda df_: df_['total_amount'] > 20]     # after filtering, 51 rows
    .shape
)

2.08 ms ± 31.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [247]:
%%timeit

(
    df                                            
    .loc[(df['passenger_count'] > 6) & (df['total_amount'] > 20)]  
)

4.09 ms ± 127 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [242]:
df.shape

(6405008, 18)

In [249]:
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [251]:
(
    df
    .fillna(0)
    .assign(passenger_count = lambda df_: df_['passenger_count'].astype(int))
)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1,1.20,1.0,N,238,239,1.0,6.00,3.00,0.5,1.47,0.00,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1,1.20,1.0,N,239,238,1.0,7.00,3.00,0.5,1.50,0.00,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1,0.60,1.0,N,238,238,1.0,6.00,3.00,0.5,1.00,0.00,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1,0.80,1.0,N,238,151,1.0,5.50,0.50,0.5,1.36,0.00,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1,0.00,1.0,N,193,193,2.0,3.50,0.50,0.5,0.00,0.00,0.3,4.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,0.0,2020-01-31 22:51:00,2020-01-31 23:22:00,0,3.24,0.0,0,237,234,0.0,17.59,2.75,0.5,0.00,0.00,0.3,21.14,0.0
6405004,0.0,2020-01-31 22:10:00,2020-01-31 23:26:00,0,22.13,0.0,0,259,45,0.0,46.67,2.75,0.5,0.00,12.24,0.3,62.46,0.0
6405005,0.0,2020-01-31 22:50:07,2020-01-31 23:17:57,0,10.51,0.0,0,137,169,0.0,48.85,2.75,0.0,0.00,0.00,0.3,51.90,0.0
6405006,0.0,2020-01-31 22:25:53,2020-01-31 22:48:32,0,5.49,0.0,0,50,42,0.0,27.17,2.75,0.0,0.00,0.00,0.3,30.22,0.0


# Exercises: Using `assign`

1. Create a data frame from the NYC taxi info in January 2020.
2. The `trip_distance` is measured in miles. Find all trips that were > 15 km in length, and which cost < 20 dollars.
3. Find all trips that cost more than 50 dollars, in which people tipped less than 10 percent, and where the number of passengers was odd.

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [256]:
(
    df
    .assign(trip_distance_km = lambda df_: df_['trip_distance'] * 1.6)  # assign is running on df
    .loc[lambda df_: df_['trip_distance_km'] > 15]                      # .loc runs on the result of assign
    .loc[lambda df_: df_['total_amount'] < 20]                          # .loc runs on the result of line 4
)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_distance_km
6497,1.0,2020-01-01 00:43:39,2020-01-01 00:44:31,1.0,21.80,1.0,N,48,48,3.0,2.50,3.00,0.5,0.0,0.00,0.3,6.30,2.5,34.880
6822,1.0,2020-01-01 00:26:48,2020-01-01 01:27:51,0.0,12.00,5.0,N,162,48,2.0,0.01,0.00,0.0,0.0,0.00,0.3,0.31,0.0,19.200
10786,2.0,2020-01-01 00:04:53,2020-01-01 00:26:25,1.0,11.35,1.0,N,107,14,4.0,-32.50,-0.50,-0.5,0.0,0.00,-0.3,-36.30,-2.5,18.160
18726,2.0,2020-01-01 01:50:41,2020-01-01 02:34:05,4.0,16.38,1.0,N,68,265,4.0,-50.00,-0.50,-0.5,0.0,0.00,-0.3,-53.80,-2.5,26.208
19225,2.0,2020-01-01 01:52:47,2020-01-01 02:17:25,1.0,10.85,1.0,N,236,18,4.0,-31.50,-0.50,-0.5,0.0,0.00,-0.3,-35.30,-2.5,17.360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6382928,NaN,2020-01-17 15:22:00,2020-01-17 16:39:00,NaN,13.43,NaN,NaN,60,53,NaN,-53.00,2.75,0.5,0.0,6.12,0.3,-43.33,0.0,21.488
6384665,NaN,2020-01-18 22:44:00,2020-01-18 23:31:00,NaN,22.86,NaN,NaN,48,118,NaN,-110.41,2.75,0.5,0.0,18.36,0.3,-88.50,0.0,36.576
6392809,NaN,2020-01-24 15:14:44,2020-01-24 17:00:41,NaN,10.53,NaN,NaN,68,129,NaN,14.59,2.75,0.0,0.0,0.00,0.3,17.64,0.0,16.848
6393652,NaN,2020-01-24 12:16:58,2020-01-24 12:46:58,NaN,10.59,NaN,NaN,76,216,NaN,15.43,2.75,0.0,0.0,0.00,0.3,18.48,0.0,16.944


In [260]:
# Find all trips that cost more than 50 dollars, in which people tipped less than 10 percent, 
# and where the number of passengers was odd.

(
    df
    .loc[lambda df_: df_['total_amount'] > 50]
    .assign(tip_percentage = lambda df_: df_['tip_amount'] / df_['fare_amount'])
    .loc[lambda df_: df_['tip_percentage'] < 0.1]
    .loc[lambda df_: df_['passenger_count'] % 2 == 1]
)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,tip_percentage
238,1.0,2020-01-01 00:21:13,2020-01-01 00:50:30,1.0,18.20,2.0,N,132,116,1.0,52.0,0.0,0.5,0.0,6.12,0.3,58.92,0.0,0.0
247,1.0,2020-01-01 00:50:55,2020-01-01 01:43:59,1.0,13.30,1.0,N,114,55,2.0,46.5,3.0,0.5,0.0,0.00,0.3,50.30,2.5,0.0
333,1.0,2020-01-01 00:40:01,2020-01-01 01:16:16,1.0,13.60,1.0,N,234,221,2.0,40.5,3.0,0.5,0.0,18.36,0.3,62.66,2.5,0.0
699,1.0,2020-01-01 00:31:32,2020-01-01 00:31:45,1.0,0.00,2.0,N,236,43,3.0,52.0,2.5,0.5,0.0,0.00,0.3,55.30,2.5,0.0
1306,2.0,2020-01-01 00:49:43,2020-01-01 00:50:46,1.0,0.00,5.0,N,265,265,1.0,45.0,0.0,0.0,0.0,10.50,0.3,55.80,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339324,2.0,2020-01-31 23:17:32,2020-01-31 23:52:26,1.0,17.90,2.0,N,132,141,2.0,52.0,0.0,0.5,0.0,0.00,0.3,55.30,2.5,0.0
6339344,2.0,2020-01-31 23:29:56,2020-01-31 23:30:29,1.0,0.00,5.0,N,261,231,2.0,65.0,0.0,0.0,0.0,0.00,0.3,67.80,2.5,0.0
6339418,2.0,2020-01-31 23:21:56,2020-01-31 23:52:38,1.0,7.20,4.0,N,100,100,2.0,40.5,0.5,0.5,0.0,11.75,0.3,56.05,2.5,0.0
6339465,2.0,2020-01-31 23:54:51,2020-01-31 23:56:50,1.0,0.05,2.0,N,132,132,2.0,52.0,0.0,0.5,0.0,0.00,0.3,52.80,0.0,0.0


# `pipe`

Method chaining is great, when you have methods to call. But sometimes, you want to call a function that takes your data frame as an argument.  `pipe` is a data frame method that reverses the direction of a call.

If you say

```python
(
    df
    .pipe(func, 5, 10)
)
```

That's the equivalent of saying `func(df, 5, 10)`.  In this way, you can still include such a function call in your method chain, and still get the results and continue with the chain.   


# Next up: Strings in Pandas

Resume: at :10

In [261]:
a = np.array('this is a bunch of strings'.split())


In [262]:
a

array(['this', 'is', 'a', 'bunch', 'of', 'strings'], dtype='<U7')

In [263]:
a[0] = 'abcdefghijkl'


In [264]:
a

array(['abcdefg', 'is', 'a', 'bunch', 'of', 'strings'], dtype='<U7')

In [265]:
s = Series('this is a bunch of strings for my course'.split())
s

0       this
1         is
2          a
3      bunch
4         of
5    strings
6        for
7         my
8     course
dtype: object

The worst thing you can do in Pandas is a `for` loop. But how else can we work with the strings in our series? If I want to calculate, or extract, or something else, it's hard to imagine how else to do it.

Fortunately, Pandas provides us with a solution, the `.str` accessor. If you take a string (`object`) column and apply `.str` to it, that gives you access to a lot of methods that can work on strings.  Many of them are the same methods that we know from Python. Some are taken from other sources and inspirations. Some have the same names, but extended functionality. And others are just special to Pandas.

In [266]:
# for example, .str.len

s.str.len()

0    4
1    2
2    1
3    5
4    2
5    7
6    3
7    2
8    6
dtype: int64

In [268]:
# let's say that I only want the strings that are longer than the mean string length

s.loc[ s.str.len() > s.str.len().mean() ]

0       this
3      bunch
5    strings
8     course
dtype: object

In [269]:
s.str.lower()

0       this
1         is
2          a
3      bunch
4         of
5    strings
6        for
7         my
8     course
dtype: object

In [270]:
s.str.upper()

0       THIS
1         IS
2          A
3      BUNCH
4         OF
5    STRINGS
6        FOR
7         MY
8     COURSE
dtype: object

In [271]:
# contains is a super-version of "in" 

s.str.contains('e')

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8     True
dtype: bool

In [273]:
# we can also use regular expressions!

s.str.contains('[eio]', regex=True)

0     True
1     True
2    False
3    False
4     True
5     True
6     True
7    False
8     True
dtype: bool

10-day e-mail series to teach you regular expressions in Python: https://RegexpCrashCourse.com 

In [274]:
s = '12345'

s.isdigit()

True

In [275]:
s = Series('a b c 5 10 15 d e f'.split())
s

0     a
1     b
2     c
3     5
4    10
5    15
6     d
7     e
8     f
dtype: object

In [276]:
s.astype(int)

ValueError: invalid literal for int() with base 10: 'a'

In [282]:
(
    s
    .loc[ lambda s_: s_.str.isdigit() ]
    .astype(int)
    .describe()
)

count     3.0
mean     10.0
std       5.0
min       5.0
25%       7.5
50%      10.0
75%      12.5
max      15.0
dtype: float64

In [283]:
s

0     a
1     b
2     c
3     5
4    10
5    15
6     d
7     e
8     f
dtype: object

In [284]:
s = Series('here are a bunch of words of varying lengths for my course'.split())
s

0        here
1         are
2           a
3       bunch
4          of
5       words
6          of
7     varying
8     lengths
9         for
10         my
11     course
dtype: object

In [285]:
# I want the first letter of each word
# normally, we would use []... but we can't do that here
# instead, we'll use .str.get which takes an integer or a slice object

s.str.get(0)

0     h
1     a
2     a
3     b
4     o
5     w
6     o
7     v
8     l
9     f
10    m
11    c
dtype: object

In [286]:
s = Series('abc-defg hi-jklm nop-qrstu'.split())
s

0     abc-defg
1      hi-jklm
2    nop-qrstu
dtype: object

In [287]:
s.str.split('-')

0     [abc, defg]
1      [hi, jklm]
2    [nop, qrstu]
dtype: object

In [289]:
# I want to get the part before the - in each of these strings
# it turns out that if you have Python objects, then you can use the .str accessor -- not just on strings!

s.str.split('-').str.get(0)

0    abc
1     hi
2    nop
dtype: object

In [291]:
# if you have a series of lists, you can turn that into a longer series, in which each list element is a series element
s.str.split('-')

0     [abc, defg]
1      [hi, jklm]
2    [nop, qrstu]
dtype: object

In [293]:
# I can run the "explode" method
# this series method returns a new series whose values are all of the individual (internal)
# values from our lists.. the indexes reflect where the elements are originally from.

s.str.split('-').explode()

0      abc
0     defg
1       hi
1     jklm
2      nop
2    qrstu
dtype: object

In [294]:
filename = '/Users/reuven/Courses/Current/Data/alice-in-wonderland.txt'

s = Series(open(filename).read().split())
s

0             ﻿The
1          Project
2        Gutenberg
3            EBook
4               of
           ...    
12758           to
12759         hear
12760        about
12761          new
12762      eBooks.
Length: 12763, dtype: object

In [295]:
s.value_counts().head(5)

the    732
and    362
a      321
to     311
of     300
Name: count, dtype: int64

In [297]:
!ls -lh $filename

-rw-r--r-- 1 reuven staff 73K Oct 12  2021 /Users/reuven/Courses/Current/Data/alice-in-wonderland.txt


# Exercises: Wine data

1. In the big zipfile, you'll find the 150,000 different reviews from Wine Magazine.
2. Find the 10 most common words that are longer than 4 letters for wines made in France.
3. Find the 10 most common words that are longer than 4 letters for wines made in California.


In [298]:
filename = '/Users/reuven/Courses/Current/Data/winemag-150k-reviews.csv'

!head $filename

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez
2,US,"Mac Watson honors the memory 

In [299]:
df = pd.read_csv(filename)
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [306]:
# Find the 10 most common words that are longer than 4 letters for wines made in France.


(
    df
    .loc[lambda df_: df_['country'] == 'France']   # filter the data frame by France (for country)
    ['description']                      # grab only the description column
    .str.split()                         # turn each description into a list of strings
    .explode()                           # turn that series of lists into a series of words
    .loc[lambda s_: s_.str.len() > 4]    # keep only the words that are > 4 letters
    .value_counts()
    .head(10)
)

description
fruit       5828
wine,       4806
acidity     4237
flavors     4181
tannins     3869
acidity.    3169
black       2974
Drink       2928
wine.       2842
fruits      2831
Name: count, dtype: int64

In [309]:
# let's get rid of punctuation

(
    df
    .loc[lambda df_: df_['country'] == 'France']   # filter the data frame by France (for country)
    ['description']                      # grab only the description column
    .str.split()                         # turn each description into a list of strings
    .explode()                           # turn that series of lists into a series of words
    .str.strip('.?!\'\",-')               # remove any/all of these characters that are on the *outside* of a string
    .loc[lambda s_: s_.str.len() > 4]    # keep only the words that are > 4 letters
    .value_counts()
    .head(10)
)

description
fruit        8675
acidity      8508
flavors      7661
tannins      6438
fruits       5449
fresh        3957
character    3494
finish       3244
texture      3067
black        2994
Name: count, dtype: int64

In [311]:
# Find the 10 most common words that are longer than 4 letters for wines made in California.

(
    df
    .loc[lambda df_: df_['province'] == 'California']   # filter the data frame by California
    ['description']                      # grab only the description column
    .str.split()                         # turn each description into a list of strings
    .explode()                           # turn that series of lists into a series of words
    .str.strip('.?!\'\",-')               # remove any/all of these characters that are on the *outside* of a string
    .loc[lambda s_: s_.str.len() > 4]    # keep only the words that are > 4 letters
    .value_counts()
    .head(10)
)

description
flavors       30504
cherry        11168
tannins       10105
acidity       10098
fruit          9901
sweet          8184
finish         7102
blackberry     6739
black          6662
cherries       5772
Name: count, dtype: int64

In [310]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [315]:
# does a longer description correlate with the number of points that a wine gets?
# we can use the "corr" method, which looks at the correlation between two numeric series -- 0 means none, -1 means 
#  perfect negative correlation, and 1 means perfect positive correlation

(
    df
    .assign(description_word_length = lambda df_: df_['description'].str.split().str.len())
    [['description_word_length', 'points', 'price']]
    .corr()
)

,description_word_length,points,price
description_word_length,1.000000,0.478487,0.238716
points,0.478487,1.000000,0.459863
price,0.238716,0.459863,1.000000
